In [1]:
#!pip install aitextgen
#!pip install sentence_transformers
#!pip install fuzzywuzzy

In [12]:
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive, copy_file_to_gdrive
from aitextgen import aitextgen
import pandas as pd
from collections import Counter

In [13]:
from sentence_transformers import SentenceTransformer
# nltk.download('wordnet')
# nltk.download('stopwords')
from sklearn.cluster import KMeans
import pandas as pd 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
class Clusters():
    def __init__(self, n=10):
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.num_clusters = n

    def train(self, txt):
        corpus_embeddings = self.embedder.encode(txt.values)
        self.kmeans = KMeans(n_clusters=self.num_clusters,init='k-means++',max_iter=300,n_init=10,random_state=0)
        self.kmeans.fit(corpus_embeddings)
        return self.kmeans.predict(corpus_embeddings)


    def test(self, txt):
        corpus_embeddings = self.embedder.encode(txt.values)
        return self.kmeans.predict(corpus_embeddings)

In [15]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [16]:
train = pd.read_json('train.json')
train['text'] = "Category: " +  train.category + " # " + train.Rationale.str.replace('"', '') + ' # ' + train.Problem
train['prompt'] = "Category: " +  train.category + " # " + train.Rationale.str.replace('"', '') + ' # '
train['clean'] = train[['text']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False,lst_stopwords=nltk.corpus.stopwords.words("english")))

train.text.to_csv('rationale_train.csv', index=False, header=None)

In [ ]:
train.shape

In [ ]:
test.shape

In [8]:
ncluster = 6
physics = Clusters(ncluster)
physics_train = physics.train(train[train.category == 'physics']['clean'])
physics_test = physics.test(train[train.category == 'physics']['clean'])

loading configuration file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "nreimers/MiniLM-L6-H384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the w

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
phys_expl = train[train.category == 'physics']
phys_expl['cluster'] = physics_test
phys_expl['prompt'] = "Category: " + phys_expl.category + " # " + phys_expl.Rationale.str.replace('"', '') + " # "

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
phys_expl

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category,text,prompt,clean,cluster
7,6 workers should finish a job in 8 days . afte...,"""let rate of one worker be r = > ( 6 * r ) * 8...","a ) 3 , b ) 4 , c ) 5 , d ) 6 , e ) 7",a,"divide(subtract(multiply(6, 8), multiply(3, 6)...","add(n0,n3)|multiply(n0,n1)|multiply(n0,n2)|sub...",physics,Category: physics # let rate of one worker be ...,Category: physics # let rate of one worker be ...,category physics let rate one worker r 6 r 8 1...,2
11,"a train 125 m long passes a man , running at 1...","""speed of the train relative to man = ( 125 / ...","a ) 60 , b ) 50 , c ) 28 , d ) 26 , e ) 29",a,"divide(divide(subtract(125, multiply(multiply(...","multiply(n1,const_0_2778)|multiply(n1,#0)|subt...",physics,Category: physics # speed of the train relativ...,Category: physics # speed of the train relativ...,category physics speed train relative man 125 ...,1
20,kim finds a 5 - meter tree branch and marks it...,"""3 pieces of 1 / 5 length and two piece each o...","a ) 8 / 15 , b ) 1 / 2 , c ) 7 / 5 , d ) 3 / 5...",d,"subtract(const_1, add(add(divide(5, multiply(a...","add(const_2,n0)|multiply(n0,#0)|divide(n0,#1)|...",physics,Category: physics # 3 pieces of 1 / 5 length a...,Category: physics # 3 pieces of 1 / 5 length a...,category physics 3 pieces 1 5 length two piece...,4
25,a garrison of 2000 men has provisions for 54 d...,"""2000 - - - - 54 2000 - - - - 39 x - - - - - 2...","a ) 1778 , b ) 1682 , c ) 9178 , d ) 1900 , e ...",d,"subtract(divide(subtract(multiply(2000, 54), m...","multiply(n0,n1)|multiply(n0,n2)|subtract(#0,#1...",physics,Category: physics # 2000 - - - - 54 2000 - - -...,Category: physics # 2000 - - - - 54 2000 - - -...,category physics 2000 54 2000 39 x 20 x 20 200...,2
28,how many seconds will a 900 meter long train m...,"""explanation : here distance d = 900 mts speed...","a ) 48 , b ) 36 , c ) 54 , d ) 11 , e ) 18",c,"divide(900, multiply(subtract(63, 3), const_0_...","subtract(n1,n2)|multiply(#0,const_0_2778)|divi...",physics,Category: physics # explanation : here distanc...,Category: physics # explanation : here distanc...,category physics explanation distance 900 mts ...,1
...,...,...,...,...,...,...,...,...,...,...,...
29825,find the area of trapezium whose parallel side...,"""area of a trapezium = 1 / 2 ( sum of parallel...","a ) 227 , b ) 299 , c ) 330 , d ) 161 , e ) 212",c,"quadrilateral_area(15, 18, 26)","quadrilateral_area(n2,n1,n0)|",physics,Category: physics # area of a trapezium = 1 / ...,Category: physics # area of a trapezium = 1 / ...,category physics area trapezium 1 2 sum parall...,4
29826,"a train 160 m long passes a man , running at 6...","""speed of train relative to man : 160 / 6 * 18...","a ) 54 kmph , b ) 60 kmph , c ) 66 kmph , d ) ...",e,"divide(divide(subtract(160, multiply(multiply(...","multiply(n1,const_0_2778)|multiply(n1,#0)|subt...",physics,Category: physics # speed of train relative to...,Category: physics # speed of train relative to...,category physics speed train relative man 160 ...,1
29827,an aeroplane covers a certain distance at a sp...,"""distance = ( 240 x 6 ) = 1440 km . speed = di...","a ) 520 , b ) 620 , c ) 820 , d ) 740 , e ) 864",e,"divide(divide(multiply(240, 6), add(const_1, d...","divide(const_2,const_3)|multiply(n0,n1)|add(#0...",physics,Category: physics # distance = ( 240 x 6 ) = 1...,Category: physics # distance = ( 240 x 6 ) = 1...,category physics distance 240 x 6 1440 km spee...,0
29828,if 9 engines consume 24 metric tonnes of coal ...,let 3 engines of former type consume 1 unit in...,"a ) 22 , b ) 24 , c ) 26 , d ) 28 , e ) none o...",c,"multiply(13, multiply(divide(multiply(3, 24), ...","multiply(n1,n5)|multiply(n0,n2)|divide(#0,#1)|...",physics,Category: physics # let 3 engines of former ty...,Category: physics # let 3 engines of former ty...,category physics let 3 engines former type con...,2


In [ ]:
phys_expl['generated'] = phys_expl.apply(lambda row: ai_0.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_expl['generated']

In [10]:
texts = []
for i in range(ncluster):
  texts.append(' '.join(phys_expl[phys_expl.cluster == i].clean.tolist()))

In [ ]:
"""
ai = aitextgen()
ai.train('rationale_train_0.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_0'
         )
"""

In [ ]:
"""
ai = aitextgen()
ai.train('rationale_train.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )
"""

In [ ]:
"""
ai_5 = aitextgen()
ai_5.train('rationale_train_5.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_5'
         )
"""

In [ ]:
"""
ai_3 = aitextgen()
ai_3.train('rationale_train_3.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_3'
         )
"""

In [ ]:
"""
ai_4 = aitextgen()
ai_4.train('rationale_train_4.csv',
         line_by_line=True,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1,
         output_dir='trained_model_4'
         )
"""

In [ ]:
test = pd.read_json('test.json')
test['prompt'] = "Category: " + test.category + " # " + test.Rationale.str.replace('"', '') + " # "
test['text'] = "Category: " + test.category + " # " + test.Rationale.str.replace('"', '') + " # " + test.Problem
test

In [ ]:
phys_test = test[test.category == 'physics']
phys_test['clean'] = phys_test[['text']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False,lst_stopwords=nltk.corpus.stopwords.words("english")))
phys_test['cluster'] = physics.test(phys_test)
phys_test_0 = phys_test[phys_test.cluster == 0]

In [5]:
#ai = aitextgen(model_folder='trained_model', to_gpu=True)
ai_0 = aitextgen(model_folder='trained_model_0', to_gpu=True)
#ai_1 = aitextgen(model_folder='trained_model_1', to_gpu=True)

In [ ]:
phys_test_0['generated'] = phys_test_0.apply(lambda row: ai_0.generate(n=5, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_0['generated']

In [48]:
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class Comparer():
    def __init__(self, l=1.0):
        self.l = l
        self.pattern = r'[0-9][0-9]*'

    def convert(self, s):
        nums = list(filter(lambda elem: elem.isnumeric(), s.split(' ')))
        new_string = re.sub(self.pattern, 'Var', s)
        return new_string, nums

    def lossy_numbers(self, l1, l2):
        loss = 0
        for i in range(min(len(l1), len(l2))):
            if l1[i] != l2[i]:
                loss += 1
        loss += max(len(l1), len(l2)) - min(len(l1), len(l2))
        return loss


    def loss_sentences(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        return fuzz.partial_ratio(m1,m2)/100.0, self.l*(self.lossy_numbers(l1, l2))


In [39]:
c = Comparer()

In [ ]:
phys_test_0['score'] = phys_test_0.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_0

In [ ]:
phys_test_0['gpt_generated'] = phys_test_0.apply(lambda row: ai.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_0['gpt_generated']

In [ ]:
phys_test_0['gpt_score'] = phys_test_0.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)
phys_test_0

In [ ]:
phys_test_0.to_csv('output_0.csv')

In [ ]:
phys_test_0 = pd.read_csv('output_0.csv').iloc[:, 1:]
phys_test_0['score'] = phys_test_0.apply(lambda row: eval(row['score']), axis=1)
phys_test_0['gpt_score'] = phys_test_0.apply(lambda row: eval(row['gpt_score']), axis=1)
phys_test_0

In [ ]:
phys_test_0[['s_score', 'n_score']] = phys_test_0.score.tolist()
phys_test_0[['s_gptscore', 'n_gptscore']] = phys_test_0.gpt_score.tolist()
print(phys_test_0.s_score.mean())
print(phys_test_0.s_gptscore.mean())
print(phys_test_0.n_score.mean())
print(phys_test_0.n_gptscore.mean())

In [ ]:
phys_test_0.to_csv('output_0.csv')

In [ ]:
phys_test['gpt_generated'] = phys_test.apply(lambda row: ai.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test['gpt_score'] = phys_test.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)

In [ ]:
phys_test.to_csv('physicsgenerated.csv')

In [ ]:
phys_test = pd.read_csv('physicsgenerated.csv')
phys_test['gpt_score'] = phys_test.apply(lambda row: eval(row['gpt_score']), axis=1)

In [ ]:
phys_test[['s_gptscore', 'n_gptscore']] = phys_test['gpt_score'].tolist()
print(phys_test.s_gptscore.mean())
print(phys_test.n_gptscore.mean())

In [ ]:
phys_test_1 = phys_test[phys_test.cluster == 1]
phys_test_1['gpt_score'] = phys_test_1.apply(lambda row: c.loss_sentences(row['text'], row['gpt_generated'][0]), axis=1)
phys_test[['s_gptscore', 'n_gptscore']] = phys_test['gpt_score'].tolist()

In [ ]:
phys_test_1['generated'] = phys_test_1.apply(lambda row: ai_1.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_1['score'] = phys_test_1.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)

In [ ]:
phys_test_1[['s_score', 'n_score']] = phys_test_1.score.tolist()
phys_test_1.to_csv('output_1.csv')
print(phys_test_1.s_score.mean())
print(phys_test_1.n_score.mean())
print(phys_test[phys_test.cluster == 1].s_gptscore.mean())
print(phys_test[phys_test.cluster == 1].n_gptscore.mean())

In [ ]:
ai_2 = aitextgen(model_folder='trained_model_2', to_gpu=True)
ai_3 = aitextgen(model_folder='trained_model_3', to_gpu=True)
ai_4 = aitextgen(model_folder='trained_model_4', to_gpu=True)
ai_5 = aitextgen(model_folder='trained_model_5', to_gpu=True)

In [ ]:
phys_test_2 = phys_test[phys_test.cluster == 2]
phys_test_2['generated'] = phys_test_2.apply(lambda row: ai_2.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_2['score'] = phys_test_2.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_2[['s_score', 'n_score']] = phys_test_2.score.tolist()
phys_test_2.to_csv('output_2.csv')
print(phys_test_2.s_score.mean())
print(phys_test_2.n_score.mean())

In [ ]:
phys_test_3 = phys_test[phys_test.cluster == 3]
phys_test_3['generated'] = phys_test_3.apply(lambda row: ai_3.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_3['score'] = phys_test_3.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_3[['s_score', 'n_score']] = phys_test_3.score.tolist()
phys_test_3.to_csv('output_3.csv')
print(phys_test_3.s_score.mean())
print(phys_test_3.n_score.mean())

In [ ]:
phys_test_4 = phys_test[phys_test.cluster == 4]
phys_test_4['generated'] = phys_test_4.apply(lambda row: ai_4.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_4['score'] = phys_test_4.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_4[['s_score', 'n_score']] = phys_test_4.score.tolist()
phys_test_4.to_csv('output_4.csv')
print(phys_test_4.s_score.mean())
print(phys_test_4.n_score.mean())

In [ ]:
phys_test_5 = phys_test[phys_test.cluster == 5]
phys_test_5['generated'] = phys_test_5.apply(lambda row: ai_5.generate(n=1, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
phys_test_5['score'] = phys_test_5.apply(lambda row: c.loss_sentences(row['text'], row['generated'][0]), axis=1)
phys_test_5[['s_score', 'n_score']] = phys_test_5.score.tolist()
phys_test_5.to_csv('output_5.csv')
print(phys_test_5.s_score.mean())
print(phys_test_5.n_score.mean())

In [ ]:
(phys_test_0.s_score.mean() + phys_test_1.s_score.mean() + phys_test_2.s_score.mean() + phys_test_3.s_score.mean() + \
 phys_test_4.s_score.mean() + phys_test_5.s_score.mean()) / 6

In [ ]:
(phys_test_0.n_score.mean() + phys_test_1.n_score.mean() + phys_test_2.n_score.mean() + phys_test_3.n_score.mean() + \
 phys_test_4.n_score.mean() + phys_test_5.n_score.mean()) / 6

In [62]:
from sentence_transformers import SentenceTransformer
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd 
import re
import nltk
import torch
import numpy as np
import tensorflow as tf
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

class Comparer():
    def __init__(self, l=1.0):
        self.l = l
        self.pattern = r'[0-9][0-9]*'

    def convert(self, s):
        nums = list(filter(lambda elem: elem.isnumeric(), s.split(' ')))
        new_string = re.sub(self.pattern, 'Var', s)
        return new_string, nums

    def lossy_numbers(self, l1, l2):
        loss = 0
        for i in range(min(len(l1), len(l2))):
            if l1[i] != l2[i]:
                loss += 1
        loss += max(len(l1), len(l2)) - min(len(l1), len(l2))
        return loss


    def loss_sentences(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        #print(fuzz.ratio(m1,m2))
        return -fuzz.partial_ratio(m1,m2)/100.0 + self.l*(self.lossy_numbers(l1, l2))
    
    def loss_sentences_output(self, s1, s2):
        m1, l1 = self.convert(s1)
        m2, l2 = self.convert(s2)
        return -fuzz.partial_ratio(m1,m2)/100.0, self.l*(self.lossy_numbers(l1, l2))

def get_val(s):
    return list(filter(lambda elem: elem.isnumeric(), s.split(' ')))

def get_label(s, l):
    return list(map(lambda elem: 2*int(elem in l)-1, s.split(' ')))

def generate_numbers(data, c):
    data['labels'] = data.apply(lambda r: c.loss_sentences(r['clean'], r['generated_clean']) , axis=1)
    #data['labels'] = np.log(data['labels'])
    data['labels'] = data['labels']/data['labels'].max()
    return data['labels']

In [18]:
output = pd.read_csv('physicsgenerated.csv')
output_0 = pd.read_csv('output_0.csv')
output_1 = pd.read_csv('output_1.csv')
output_2 = pd.read_csv('output_2.csv')
output_3 = pd.read_csv('output_3.csv')
output_4 = pd.read_csv('output_4.csv')
output_5 = pd.read_csv('output_5.csv')

In [19]:
# Build a model
inputs = layers.Input(shape=(384,))
layer1 = layers.Dense(128, activation='relu')(inputs)
layer2 = layers.Dense(128)(layer1)
layer3 = keras.layers.LeakyReLU(alpha=0.3)(layer2)
predictions = layers.Dense(1, activation='sigmoid')(layer3)
model = keras.Model(inputs=inputs, outputs=predictions)
print('Model Built')
# Define custom loss

2021-12-07 03:40:51.990419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 03:40:52.003317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 03:40:52.004082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model Built


2021-12-07 03:40:52.006635: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-07 03:40:52.007260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 03:40:52.007966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 03:40:52.008714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [20]:
output_0['generated']

0      ['Category: physics # distance covered by a in...
1      ['Category: physics # records last 90 min on e...
2      ['Category: physics # explanation : | - - - - ...
3      ['Category: physics # the ratio is 2 : 3 = gym...
4      ['Category: physics # in 36 minutes, car x tra...
                             ...                        
175    ["Category: physics # here, drawing a quick sk...
176    ['Category: physics # the total distance d tra...
177    ["Category: physics # solution let us name the...
178    ["Category: physics # number of stops in an ho...
179    ['Category: physics # 35 * 18 / 5 = 126 kmph a...
Name: generated, Length: 180, dtype: object

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean']

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_0, Comparer())
output_0.to_csv('output_0.csv')

output_1['generated_clean'] = output_1[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_1, Comparer())
output_1.to_csv('output_1.csv')

output_2['generated_clean'] = output_2[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_2, Comparer())
output_2.to_csv('output_2.csv')

output_3['generated_clean'] = output_3[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_3, Comparer())
output_3.to_csv('output_3.csv')

output_4['generated_clean'] = output_4[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_4, Comparer())
output_4['loss'] = labels
output_4.to_csv('output_4.csv')

output_5['generated_clean'] = output_5[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_5, Comparer())
output_5.to_csv('output_5.csv')

output['generated_clean'] = output[['gpt_generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output, Comparer())
output.to_csv('output.csv')

In [ ]:
output_0['generated_clean'] = output_0[['generated']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
labels = generate_numbers(output_0, Comparer())
output_0.to_csv('output_0.csv')

# Compile the model
model.compile(optimizer='adam',
              loss='mse', # Call the loss function with the selected layer
              metrics=['accuracy'])
print('Model  compiled')
embedder = SentenceTransformer('all-MiniLM-L6-v2')
model.fit(embedder.encode(output_0['generated_clean'].values), np.array(labels))

In [9]:
import keras

In [21]:
output_0 = pd.read_csv('output_0.csv')

In [29]:
output_0['generated'] = output_0.apply(lambda row: ai_0.generate(n=5, prompt=row['prompt'], repetition_penalty=1.1, return_as_list=True, max_length=500), axis=1)
output_0[['generated_0', 'generated_1', 'generated_2', 'generated_3', 'generated_4']] = \
    output_0['generated'].tolist()

In [58]:
output_0['generated_clean_0'] = output_0[['generated_0']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_1'] = output_0[['generated_1']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_2'] = output_0[['generated_2']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_3'] = output_0[['generated_3']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))
output_0['generated_clean_4'] = output_0[['generated_4']].applymap(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=nltk.corpus.stopwords.words("english")))

In [32]:
output_0.to_csv('output_0.csv')

In [35]:
output_0['generated_clean_0'][0]

'category physic distance covered 30 min 1 km b cover extra 1 km 1 hour 48 minute 9 5 hr e relative speed b 1 9 5 5 9 speed b speed 5 9 6 5 9 655 answer b т drove town town b speed 30 km hr 30 minute speed reduced 1 km hr distance speed km hr town go town b'

In [59]:
model = keras.models.load_model('.')
embedder = SentenceTransformer('all-MiniLM-L6-v2')

min_score = 10000
min_col = ''
for col in ['generated_clean_0', 'generated_clean_1', 'generated_clean_2', 'generated_clean_3', 'generated_clean_4']:
    col_name = col + '_score'
    output_0[col_name] = model.predict(embedder.encode(output_0[col].values))
cols = ['generated_clean_0_score', 'generated_clean_1_score', 'generated_clean_2_score', 'generated_clean_3_score', 'generated_clean_4_score']
output_0['min_col'] = output_0[cols].idxmin(axis=1)
output_0['min_col'] = output_0.apply(lambda x: x['min_col'].strip('_score'), axis=1)
output_0['output'] = output_0.apply(lambda x: x[x['min_col']], axis=1)
output_0['output']

Didn't find file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/added_tokens.json. We won't load it.
loading file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/vocab.txt
loading file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/tokenizer.json
loading file None
loading file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/special_tokens_map.json
loading file /home/jupyter/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/tokenizer_config.json


0      category physic distance covered 30 min 1 km b...
1      category physic record last 90 min 2 side reco...
2      category physic explanation 20 60 50 20 r 60 5...
3      category physic ratio 2 3 gym ride 10 3 2 15 m...
4      category physic 36 minute car x travel 21 mile...
                             ...                        
175    category physic drawing quick sketch theaction...
176    category physic total distance traveled john g...
177    category physic solution let u name brother b ...
178    category physic number stop hour 60 5 12 dista...
179          category physic 35 18 5 126 kmph answer c ɵ
Name: output, Length: 180, dtype: object

In [37]:
output_0.to_csv('output_0.csv')

In [63]:
c = Comparer()

In [64]:
output_0['score'] = output_0.apply(lambda row: c.loss_sentences_output(row['clean'], row['output']), axis=1)
output_0[['s_score', 'n_score']] = output_0.score.tolist()
output_0.to_csv('output_0.csv')
print(output_0.s_score.mean())
print(output_0.n_score.mean())

-0.5844999999999997
4.461111111111111


In [53]:
s1 = "a rectangular floor that measures 15 meters by 18 meters is to be covered with carpet squares that each measure 10 meters by 3 meters. if the carpet squares cost $ 12 apiece, how much will the carpet squares cost?"
s2 = "a rectangular floor that measures 15 meters by 18 meters is to be covered with carpet squares that each measure 3 meters by 3 meters . if the carpet squares cost $12 apiece , what is the total cost for the number of carpet squares needed to cover the floor ?"
c.loss_sentences(s1,s2)

(0.83, 2.0)